In [1]:
# https://www.datacamp.com/tutorial/fine-tuning-llama-3-1

In [1]:
import numpy as np
import pandas as pd
import wandb
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset,load_dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)

/home/alopez/anaconda3/envs/llm_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-05 12:01:55.257912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 12:01:55.271010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 12:01:55.274957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 12:01:55.285524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlo

### 1.Loading and processing dataset

In [ ]:
# PIMA
train_dataset = pd.read_csv('./PIMA_dataset/train_data.csv')
validation_dataset = pd.read_csv('./PIMA_dataset/validation_data.csv')
test_dataset = pd.read_csv('./PIMA_dataset/test_data.csv')
print("Train dataset shape",train_dataset.shape)
print("Validation dataset shape",validation_dataset.shape)
print("Test dataset shape",test_dataset.shape)

target_column = "Outcome"

Train dataset shape (491, 9)
Validation dataset shape (123, 9)
Test dataset shape (154, 9)


In [ ]:
# Define the prompt generation functions
def generate_prompt(row):
    features_text = " ".join([
        f"The {col} is {str(row[col])}." for col in feature_columns
    ])
    # PIMA
    prompt = f"""You are a doctor specialised in classifying patients as diabetic or non-diabetic based on their health values. Instruction: Respond only with '0' for non-diabetic or '1' for diabetic. Use the following output format: 'Outcome: 0'.
    Predict the {target_column}. Health values: {features_text}.
    Outcome: '{int(row[target_column])}'."""
    return prompt

def generate_test_prompt(row):
    features_text = " ".join([
        f"The {col} is {str(row[col])}." for col in feature_columns
    ])
    # PIMA
    prompt = f"""You are a doctor specialised in classifying patients as diabetic or non-diabetic based on their health values. Instruction: Respond only with '0' for non-diabetic or '1' for diabetic. Use the following output format: 'Outcome: 0'.
    Predict the {target_column}. Health values: {features_text}.
    Outcome: """.strip()
    return prompt


feature_columns = [col for col in train_dataset.columns if col != target_column]
train_dataset.loc[:,"text"] = train_dataset.apply(generate_prompt,axis=1)
validation_dataset.loc[:,'text'] = validation_dataset.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
test_dataset = pd.DataFrame(test_dataset.apply(generate_test_prompt, axis=1), columns=["text"])

# Convert to HuggingFace datasets
train_data = Dataset.from_pandas(train_dataset[["text"]])
eval_data = Dataset.from_pandas(validation_dataset[["text"]])

In [4]:
train_data['text'][3]

"You are a doctor specialised in classifying patients as diabetic or non-diabetic based on their health values. Instruction: Respond only with '0' for non-diabetic or '1' for diabetic. Use the following output format: 'Outcome: 0'.\n    Predict the Outcome. Health values: The Pregnancies is 2.0. The Glucose is 99.0. The BloodPressure is 52.0. The SkinThickness is 15.0. The Insulin is 94.0. The BMI is 24.6. The DiabetesPedigreeFunction is 0.637. The Age is 21.0..\n    Outcome: '0'."

### 2.Load the model and tokenizer

In [3]:
# Llama 3.1-8B-Instructur in 4bit quantization
# Llama 3.1-70B-Instructur
base_model_name = "meta-llama/Llama-3.1-70B-Instruct"
#output_dir = "Llama-3.1-405B-Instruct-Pima-Diabetes-Clasification"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

### 3.Setting up the model

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['k_proj', 'v_proj', 'up_proj', 'down_proj', 'gate_proj', 'q_proj', 'o_proj']

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,                         
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/home/alopez/anaconda3/envs/llm_env/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/alopez/anaconda3/envs/llm_env/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/alopez/anaconda3/envs/llm_env/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/alopez/anaconda3/envs/llm_env/lib/python3.12/site-packages/trl/trainer/sft_train

### 4.Model training

In [ ]:
trainer.train() 

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: andrealopezg_ (andrealopezg_-universidad-rey-juan-carlos). Use `wandb login --relogin` to force relogin


/home/alopez/anaconda3/envs/llm_env/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
13,0.256000,0.251055
26,0.253700,0.242107
39,0.233600,0.242534
52,0.242000,0.240042


TrainOutput(global_step=61, training_loss=0.36761759343694467, metrics={'train_runtime': 2778.7714, 'train_samples_per_second': 0.177, 'train_steps_per_second': 0.022, 'total_flos': 2.862432056102093e+16, 'train_loss': 0.36761759343694467, 'epoch': 0.9938900203665988})

In [ ]:
wandb.finish()
model.config.use_cache = True

eval/loss,█▂▃▁
eval/runtime,▁▆█▂
eval/samples_per_second,█▁▁█
eval/steps_per_second,▁▁▁▁
train/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▇▇▇██▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▅█████████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train/loss,█▇▆▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.24004
eval/runtime,60.3829


In [11]:
# Save trained model and tokenizer
# trainer.save_model(output_dir)
# tokenizer.save_pretrained(output_dir)

In [ ]:
trainer.model.push_to_hub(output_dir, use_temp_dir=False)

adapter_model.safetensors: 100%|██████████| 3.31G/3.31G [01:45<00:00, 31.5MB/s]


CommitInfo(commit_url='https://huggingface.co/andrealopez/Llama-3.1-70B-Instruct-Pima-Diabetes-Clasification/commit/d3ebd772e4f325e929a2afa8462dd5096d190a2f', commit_message='Upload model', commit_description='', oid='d3ebd772e4f325e929a2afa8462dd5096d190a2f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/andrealopez/Llama-3.1-70B-Instruct-Pima-Diabetes-Clasification', endpoint='https://huggingface.co', repo_type='model', repo_id='andrealopez/Llama-3.1-70B-Instruct-Pima-Diabetes-Clasification'), pr_revision=None, pr_num=None)